# v2_tratado - Análise preliminar

Neste documento serão trabalhadas as análises de negócio que o dataset permitir a extração

## Data Understanding


### Loading Libraries

In [ ]:
# Loading libraries

import pandas as pd
import numpy as np
from datetime import datetime


In [ ]:
# Reading dataset

df = pd.read_csv('/content/dataset_v2_tratado.csv', delimiter=',')

FileNotFoundError: ignored

In [ ]:
display(df)

In [ ]:
# Understanding data

df.info()

In [ ]:
# Explore missing values

missing_values = df.isna().sum()
missing_values = pd.DataFrame(missing_values,columns=['count']).reset_index()
missing_values = missing_values[missing_values['count'] != 0]
missing_values.sort_values('count',ascending=False, inplace=True)

missing_values_list = missing_values['index'].to_list()
missing_values_list

### Verifying the anonimized data

In [ ]:
#Verificando existência de padrão dos dados anonimizados

anonimized = ['instagram_hash_bio',
              'instagram_hash_bio_link',
              'instagram_hash_foto_perfil',
              'instagram_hash_public_email',
              'instagram_hash_public_whatsapp',
              'lead_hash_email',
              'lead_hash_instagram',
              'lead_hash_nome',
              'lead_hash_nome_loja',
              'lead_hash_whatsapp',
              'user_hash_instagram',
              'user_hash_whatsapp'
]

for a in anonimized:
  group_a = df.groupby(a).agg({'id':'count','user_val_cac':'mean'}).reset_index().rename(columns={'id':'count','user_val_cac':'user_val_cac_mean'}).sort_values('count',ascending=False)
  
  display(group_a)
  print('------------------------------')

### Relation between categories/CAC


In [ ]:
# Relação das categorias com média de CAC

categorical_columns = [
    'lead_fluxo', 'lead_tamanho_negocio', 'lead_utm_source', 'lead_utm_medium', 'lead_utm_campaign', 'lead_utm_content', 'lead_lp_version', 'lead_referrer', 
    'lead_city', 'lead_region', 'instagram_public_zip', 'instagram_category_bussiness', 'user_status', 'user_plano', 'user_payment', 'user_cidade', 'user_estado',
    'user_p1', 'user_p2', 'user_p3', 'user_p4', 'user_diagnostico']
dataset_cat_col = []
df_cat_col = pd.DataFrame(dataset_cat_col, columns=['cat_columns'])

for c in categorical_columns:
  group_c = df.groupby(c).agg({'user_val_cac':'mean'}).sort_values('user_val_cac',ascending=True).rename(columns={'user_val_cac':'user_val_cac_mean'}).reset_index()
  display(group_c)
  print('------------------------------------------------------')
  df_cat_col = df_cat_col.append(group_c)
  
df_cat_col.to_excel('cat_append.xlsx')
  




In [ ]:
# Avaliação CAC
df['user_val_cac'].describe()


In [ ]:
df_CAC = df['user_val_cac'].sort_values(ascending=False).to_frame()
display(df_CAC)

Tendência de os top_1 anonimizados serem valores nulos, visto que que há um CAC válido atrelado ao hash

## Data Preparation

### Converting to datetime

In [ ]:
# Converting to datetime the dates features

dates = ['lead_dt_cadastro',
          'instagram_dt_criacao_estimada', 
          'user_dt_trial', 
          'user_dt_cliente',
          'user_dt_churn_prevista',
          'user_val_inadimplencia',
          'user_dt_inadimplencia_inicial', 
          'user_dt_lead',
          'user_dt_diagnostico'
          ]

for d in dates:
  df[d] = pd.to_datetime(df[d], format="%Y/%m/%d")


### Converting to category

In [ ]:
# Converting categorical features to category

categories = ['lead_fluxo',                  
              'lead_tamanho_negocio',        
              'lead_utm_source',             
              'lead_utm_medium',             
              'lead_utm_campaign',           
              'lead_utm_content',            
              'lead_lp_version',             
              'lead_referrer',               
              'lead_city',                   
              'lead_region',                
              'instagram_public_zip',        
              'instagram_category_bussiness',
              'user_status',                 
              'user_plano',                  
              'user_payment',               
              'user_cidade',                 
              'user_estado',                 
              'user_p1',                     
              'user_p2',                    
              'user_p3',                     
              'user_p4',                    
              'user_diagnostico',            
              'lead_faixa_idade']

for c in categories:
  df[c] = df[c].astype('category')

In [ ]:
df.info()

### Converting anonimized data to Nan

In [ ]:
# Converting '9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0' hash to Nan

df[anonimized] = df[anonimized].replace('9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0', np.nan)
print(df[anonimized].isna().sum())


In [ ]:
df[anonimized].head()

### Select Data

In [ ]:
# Dropping hash features

anonimized_modified = ['instagram_hash_bio',
                       'instagram_hash_bio_link',
                       'instagram_hash_foto_perfil',
                       'instagram_hash_public_email',
                       'instagram_hash_public_whatsapp',
                       'lead_hash_instagram',
                       'lead_hash_nome',
                       'lead_hash_nome_loja',
                       'lead_hash_whatsapp',
                       'user_hash_instagram',
                       'user_hash_whatsapp'    
]

df_hash_modified = df.drop(anonimized_modified,axis=1)
print(df_hash_modified.isna().sum().sort_values(ascending=False))
print(df_hash_modified.columns)